In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

import sys
# -----------------------------
# Ajouter le repo au Python Path
# -----------------------------
sys.path.append("/Workspace/Users/mandu543@gmail.com/databricks_flights")

from src.Communs.utils import *

In [0]:
df = spark.sql(f"SELECT * FROM {SILVER_ZONE}.silver_flights_flights")
display(df)

## Créer une alimentation Dynamique vers Gold Layer

#### Parameters

Pas besoin de paramètres car multi trading et non loop 

In [0]:
# Key Cols List
key_cols_list = "['flight_id']" # eval transforme en liste
key_cols_list = eval(key_cols_list)
# CDC Column
cdc_column = "modifiedDate"

# BACK-DATED REFRESH
back_dated_refresh = ""

# Source Object 
source_object = "silver_flights_flights"

# Source Schema
source_schema = SILVER_ZONE

# Target Object
target_object = "fact_flights_flights"

# Targer Schema 
target_schema = GOLD_ZONE

# Name Col NB Record BeforeInsert
colLastColBeforeInsert = "DimNBFlightsBeforeInsert"


## INCREMENTAL DATA INGESTION

### Last Load Date

In [0]:
# Without backdated refresh
if len(back_dated_refresh) == 0:
  
  # If Table Destination exists => get last load date
  if spark.catalog.tableExists(f"{target_schema}.{target_object}") :

    last_load = spark.sql(f"SELECT max({cdc_column}) FROM {target_schema}.{target_object}").collect()[0][0]

  else: 

    last_load = "1900-01-01 00:00:00"

# backdated refresh exists   
else : 
  last_load = back_dated_refresh

# test result 
last_load

In [0]:
# Read Data After Specific Date
df_source = spark.sql(f"SELECT * FROM {source_schema}.{source_object} WHERE {cdc_column} >= '{last_load}'")

In [0]:
df_source.display()

### OLD VS NEW RECORDS

In [0]:
if spark.catalog.tableExists(f"{target_schema}.{target_object}") :
    
    # Key Columns String For Incremental
    key_cols_string_incremental = ', '.join(key_cols_list)
    # Data Already Exists
    df_target = spark.sql(f"SELECT {key_cols_string_incremental}, {colLastColBeforeInsert}, create_date, update_date FROM {target_schema}.{target_object}")

else :

    # Key Columns String For Initial
    key_cols_string_initial = [f"'' as {i}" for i in key_cols_list]
    key_cols_string_initial = ', '.join(key_cols_string_initial)
    key_cols_string_initial

    df_target = spark.sql(f"SELECT {key_cols_string_initial}, '' as {colLastColBeforeInsert}, '' as create_date, '' as update_date WHERE 1=0")

df_target.display()

In [0]:
join_condition = ' AND '.join([f"src.{i} = target.{i}" for i in key_cols_list])

In [0]:
df_source.createOrReplaceTempView("src")
df_target.createOrReplaceTempView("target")

df_join = spark.sql(f"""
                    SELECT src.*,
                            target.{colLastColBeforeInsert}, 
                            target.create_date, 
                            target.update_date 
                    FROM src 
                    LEFT JOIN target 
                    ON {join_condition} 
                """)

In [0]:
df_join.display()

In [0]:
# DF OLD RECORDS
df_old = df_join.filter(col(f"{colLastColBeforeInsert}").isNotNull())
df_old.display()

# DF NEW RECORDS
df_new = df_join.filter(col(f"{colLastColBeforeInsert}").isNull())
df_new.display()


### PREPARING DF_OLD